In [2]:
import json


config_settings = json.loads(open('../../.vscode/settings.json').read())["sqltools.connections"][0]
print(json.dumps(config_settings, indent=2))

{
  "previewLimit": 50,
  "server": "localhost",
  "port": 5432,
  "driver": "PostgreSQL",
  "name": "kafka-test",
  "database": "cta",
  "username": "cta_admin",
  "password": "chicago"
}


In [3]:
print(config_settings["server"] + ":" + str(config_settings["port"]))

"http://localhost:9021/clusters/MkU3OEVBNTcwNTJENDM2Qg/management/connect/connect-default/connectors/browse" 

localhost:5432


'http://localhost:9021/clusters/MkU3OEVBNTcwNTJENDM2Qg/management/connect/connect-default/connectors/browse'

In [4]:
"""Configures a Kafka Connector for Postgres Station data"""
import json
import logging

import requests
import urllib3

logger = logging.getLogger(__name__)


KAFKA_CONNECT_URL = "http://localhost:8083/connectors"
CONNECTOR_NAME = "stations"


"""Starts and configures the Kafka Connect connector"""
logging.debug("creating or updating kafka connect connector...")

resp = requests.get(f"{KAFKA_CONNECT_URL}/{CONNECTOR_NAME}")
print(resp.content)
if resp.status_code == 200:
    logging.debug("connector already created skipping recreation")
    


b'{"error_code":404,"message":"Connector stations not found"}'


In [10]:

# TODO: Complete the Kafka Connect Config below.
# Directions: Use the JDBC Source Connector to connect to Postgres. Load the `stations` table
# using incrementing mode, with `stop_id` as the incrementing column name.
# Make sure to think about what an appropriate topic prefix would be, and how frequently Kafka
# Connect should run this connector (hint: not very often!)

# load the configs from vs settings
config_settings = json.loads(open('../../.vscode/settings.json').read())["sqltools.connections"][0]
print(config_settings)

resp = requests.post(
    KAFKA_CONNECT_URL,
    headers={"Content-Type": "application/json"},
    data=json.dumps({
        "name": CONNECTOR_NAME,
        "config": {
            "connector.class": "io.confluent.connect.jdbc.JdbcSourceConnector",
            "key.converter": "org.apache.kafka.connect.json.JsonConverter",
            "key.converter.schemas.enable": "false",
            "value.converter": "org.apache.kafka.connect.json.JsonConverter",
            "value.converter.schemas.enable": "false",
            "batch.max.rows": "500",
            "connection.url": "jdbc:postgresql://host.docker.internal:5432/cta",
            "connection.user": config_settings["username"],
            "connection.password": config_settings["password"],
            "table.whitelist": "stations",
            "mode": "incrementing",
            "incrementing.column.name": "stop_id",
            "topic.prefix": "stations",
            "poll.interval.ms": "100",
        }
    }),
)

## Ensure a healthy response was given
print(resp.content)
resp.raise_for_status()
logging.debug("connector created successfully")

#confluent-hub install confluentinc/kafka-connect-jdbc:10.7.1
#confluent-hub install confluentinc/kafka-connect-jdbc:latest


{'previewLimit': 50, 'server': '127.0.0.1', 'port': 5432, 'driver': 'PostgreSQL', 'name': 'kafka', 'database': 'cta', 'username': 'cta_admin', 'password': 'chicago'}
b'{"name":"stations","config":{"connector.class":"io.confluent.connect.jdbc.JdbcSourceConnector","key.converter":"org.apache.kafka.connect.json.JsonConverter","key.converter.schemas.enable":"false","value.converter":"org.apache.kafka.connect.json.JsonConverter","value.converter.schemas.enable":"false","batch.max.rows":"500","connection.url":"jdbc:postgresql://127.0.0.1:5432/cta","connection.user":"cta_admin","connection.password":"chicago","table.whitelist":"stations","mode":"incrementing","incrementing.column.name":"stop_id","topic.prefix":"stations","poll.interval.ms":"100","name":"stations"},"tasks":[],"type":"source"}'
